<a href="https://colab.research.google.com/github/RajaHairi/PowerShell/blob/master/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [2]:
!pip install -U -q "google-generativeai>=0.8.2"

In [3]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkdlbmVyYXRlIGEgc2NhdmVuZ2VyIGh1bnQgZm9yIHN0cmVldCBmb29kIGFyb3VuZCB0aGUgY2l0eSBvZiBTZW91bCwgS29yZWEuIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiIyMgU2VvdWwgU3RyZWV0IEZvb2QgU2NhdmVuZ2VyIEh1bnQ6IEEgVGFzdGUgb2YgdGhlIENpdHkhXFxuXFxuKipJbnRyb2R1Y3Rpb246KiogIEdldCByZWFkeSBmb3IgYSBkZWxpY2lvdXMgYWR2ZW50dXJlIHRocm91Z2ggU2VvdWwncyB2aWJyYW50IHN0cmVldCBmb29kIHNjZW5lISBUaGlzIHNjYXZlbmdlciBodW50IHdpbGwgdGFrZSB5b3UgdG8gc29tZSBvZiB0aGUgY2l0eSdzIGJlc3QgaGlkZGVuIGdlbXMgYW5kIGljb25pYyBmb29kIHN0YWxscy4gIENvbXBsZXRlIHRoZSBjaGFsbGVuZ2VzIHRvIHVubG9jayB0aGUgdWx0aW1hdGUgc3RyZWV0IGZvb2QgZXhwZXJpZW5jZSFcXG5cXG4qKlJ1bGVzOioqXFxuXFxuKiBXb3JrIGluIHRlYW1zIG9mIDItNCBwZW9wbGUuXFxuKiBVc2UgcHVibGljIHRyYW5zcG9ydGF0aW9uIG9yIHdhbGsgdG8gcmVhY2ggdGhlIGxvY2F0aW9ucy4gIChDb25zaWRlciBzYWZldHkgYW5kIHRpbWUgY29uc3RyYWludHMpLlxcbiogQWxsIHBob3RvcyBtdXN0IGJlIGNsZWFyIGFuZCBzaG93IHlvdXIgdGVhbSB3aXRoIHRoZSBmb29kIGl0ZW0uXFxuKiBZb3UgbXVzdCB0YXN0ZS10ZXN0IGV2ZXJ5IGl0ZW0gdG8gdmVyaWZ5IGNvbXBsZXRpb24uXFxuKiBUaGUgdGVhbSB3aXRoIHRoZSBtb3N0IGNvbXBsZXRlZCBjaGFsbGVuZ2VzIHdpdGhpbiB0aGUgYWxsb3R0ZWQgdGltZSB3aW5zIVxcblxcbioqVGltZSBMaW1pdDoqKiAzIGhvdXJzXFxuXFxuKipDaGFsbGVuZ2VzOioqXFxuXFxuKiooMSkgVGhlIFNwaWN5IFN0YXJ0OioqXFxuXFxuKiAqKkNoYWxsZW5nZToqKiBGaW5kIGEgc3RyZWV0IHZlbmRvciBzZWxsaW5nIFR0ZW9rYm9ra2kgKHNwaWN5IHJpY2UgY2FrZXMpLiAgXFxuKiAqKlBob3RvIEV2aWRlbmNlOioqICBBIHBob3RvIG9mIHlvdXIgdGVhbSB3aXRoIGEgc3RlYW1pbmcgcGxhdGUgb2YgVHRlb2tib2traS5cXG4qICoqQm9udXM6KiogIElkZW50aWZ5IHRoZSB0eXBlIG9mIGNoaWxpIHBhc3RlIHVzZWQgKEdvY2h1amFuZyBvciBHb2NodWdhcnUpIGFuZCB3cml0ZSBpdCBkb3duLlxcblxcbioqKDIpIFRoZSBTd2VldCBFc2NhcGU6KipcXG5cXG4qICoqQ2hhbGxlbmdlOioqIExvY2F0ZSBhIHN0YWxsIHNlbGxpbmcgSG90dGVvayAoc3dlZXQgcGFuY2FrZXMpLiAgXFxuKiAqKlBob3RvIEV2aWRlbmNlOioqICBBIHBob3RvIG9mIHlvdXIgdGVhbSBiaXRpbmcgaW50byBhIHdhcm0gSG90dGVvaywgc2hvd2luZyB0aGUgbWVsdHkgZmlsbGluZy5cXG4qICoqQm9udXM6KiogV2hhdCBmaWxsaW5nIGRpZCB5b3UgY2hvb3NlIChlLmcuLCBjaW5uYW1vbiBzdWdhciwgYnJvd24gc3VnYXIsIGV0Yy4pP1xcblxcbioqKDMpIFRoZSBTYXZvcnkgU2Vuc2F0aW9uOioqXFxuXFxuKiAqKkNoYWxsZW5nZToqKiBGaW5kIGEgc3RyZWV0IHZlbmRvciBzZWxsaW5nIEtpbWJhcCAoc2Vhd2VlZCByaWNlIHJvbGxzKS5cXG4qICoqUGhvdG8gRXZpZGVuY2U6KiogQSBwaG90byBzaG93aW5nIHlvdXIgdGVhbSBob2xkaW5nIGF0IGxlYXN0IHRocmVlIGRpZmZlcmVudCBraW5kcyBvZiBLaW1iYXAuXFxuKiAqKkJvbnVzOioqIExpc3QgdGhlIHRocmVlIGRpZmZlcmVudCBmaWxsaW5ncyB5b3UgaWRlbnRpZmllZC5cXG5cXG4qKig0KSBUaGUgQ3Jpc3B5IENydW5jaDoqKlxcblxcbiogKipDaGFsbGVuZ2U6KiogTG9jYXRlIGEgdmVuZG9yIHNlbGxpbmcgRGFra29jaGkgKGdyaWxsZWQgc2tld2VycyBvZiBtYXJpbmF0ZWQgY2hpY2tlbikuXFxuKiAqKlBob3RvIEV2aWRlbmNlOioqIEEgcGhvdG8gb2YgeW91ciB0ZWFtIGVuam95aW5nIERha2tvY2hpLCBzaG93aW5nIHRoZSBwZXJmZWN0bHkgZ3JpbGxlZCB0ZXh0dXJlLlxcbiogKipCb251czoqKiBXaGF0IG1hcmluYWRlIGZsYXZvcnMgZGlkIHlvdSBpZGVudGlmeSAoZS5nLiwgc3dlZXQsIHNwaWN5LCBzb3kgc2F1Y2UgYmFzZWQpP1xcblxcbioqKDUpIFRoZSBSZWZyZXNoaW5nIFJldHJlYXQ6KipcXG5cXG4qICoqQ2hhbGxlbmdlOioqIEZpbmQgYSBzdGFsbCBzZWxsaW5nIEJpbmdzdSAoc2hhdmVkIGljZSBkZXNzZXJ0KS5cXG4qICoqUGhvdG8gRXZpZGVuY2U6KiogQSBwaG90byBvZiB5b3VyIHRlYW0gd2l0aCBhIGNvbG9yZnVsIEJpbmdzdSwgc2hvd2luZyB0aGUgdmFyaWV0eSBvZiB0b3BwaW5ncy5cXG4qICoqQm9udXM6KiogIExpc3QgYXQgbGVhc3QgdGhyZWUgdG9wcGluZ3Mgb24geW91ciBCaW5nc3UuXFxuXFxuXFxuKiooNikgVGhlIEhpZGRlbiBHZW06KipcXG5cXG4qICoqQ2hhbGxlbmdlOioqICBGaW5kIGEgc3RyZWV0IGZvb2QgaXRlbSBOT1QgbGlzdGVkIGFib3ZlIChyZXNlYXJjaCBiZWZvcmVoYW5kIGZvciBpbnNwaXJhdGlvbiEpLlxcbiogKipQaG90byBFdmlkZW5jZToqKiBBIHBob3RvIG9mIHlvdXIgdGVhbSB3aXRoIHRoZSBteXN0ZXJ5IGZvb2QgaXRlbSBhbmQgYSBkZXNjcmlwdGlvbiBvZiBpdCAobmFtZSBhbmQgaW5ncmVkaWVudHMpLlxcbiogKipCb251czoqKiBGaW5kIGEgbG9jYWwgcmVjb21tZW5kYXRpb24gZm9yIHRoaXMgaGlkZGVuIGdlbSAoYXNrIGEgdmVuZG9yIG9yIHBhc3NlcmJ5KS5cXG5cXG4qKig3KSBUaGUgVWx0aW1hdGUgU2VvdWwgU2VsZmllOioqXFxuXFxuKiAqKkNoYWxsZW5nZToqKiBUYWtlIGEgZnVuLCBjcmVhdGl2ZSBzZWxmaWUgd2l0aCB5b3VyIHRlYW0gaW4gZnJvbnQgb2YgYSBmYW1vdXMgU2VvdWwgbGFuZG1hcmsgd2hpbGUgZW5qb3lpbmcgYW55IG9mIHRoZSBzdHJlZXQgZm9vZCB5b3UgZm91bmQuXFxuKiAqKlBob3RvIEV2aWRlbmNlOioqIFRoZSBzZWxmaWUgc2hvd2Nhc2luZyB0aGUgbGFuZG1hcmsgYW5kIHN0cmVldCBmb29kLlxcblxcblxcbioqU2NvcmluZzoqKlxcblxcbiogRWFjaCBjb21wbGV0ZWQgY2hhbGxlbmdlOiAxMCBwb2ludHNcXG4qIEVhY2ggYm9udXMgY2hhbGxlbmdlOiA1IHBvaW50c1xcblxcbioqV2lubmluZzoqKiBUaGUgdGVhbSB3aXRoIHRoZSBoaWdoZXN0IHNjb3JlIGF0IHRoZSBlbmQgb2YgdGhlIDMtaG91ciB0aW1lIGxpbWl0IHdpbnMgYnJhZ2dpbmcgcmlnaHRzIGFuZCB0aGUgdGl0bGUgb2YgXFxcIlNlb3VsIFN0cmVldCBGb29kIENoYW1waW9ucyFcXFwiXFxuXFxuXFxuKipOb3RlOioqICBUaGlzIHNjYXZlbmdlciBodW50IGNhbiBiZSBhZGFwdGVkIHRvIHN1aXQgZGlmZmVyZW50IHNraWxsIGxldmVscyBhbmQgdGltZSBjb25zdHJhaW50cy4gWW91IGNhbiBhZGQgbW9yZSBjaGFsbGVuZ2VzLCBzcGVjaWZ5IGxvY2F0aW9ucyAoZS5nLiwgTXllb25nZG9uZywgTmFtZGFlbXVuIE1hcmtldCksIG9yIGFkanVzdCB0aGUgZGlmZmljdWx0eSBiYXNlZCBvbiB0aGUgcGFydGljaXBhbnRzJyBleHBlcmllbmNlLiBSZW1lbWJlciB0byBwcmlvcml0aXplIHNhZmV0eSBhbmQgaGF2ZSBmdW4hICBFbmpveSB0aGUgZGVsaWNpb3VzIGFkdmVudHVyZSFcXG4ifV19XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

TimeoutException: Requesting secret GOOGLE_API_KEY timed out. Secrets can only be fetched when running from the Colab UI.

## Call `generate_content`

In [ ]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")